# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [2]:
from utils import setup_db_context
import pandas as pd
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking

데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [3]:
# input data 예시 "지역구" = 도봉구, "법정동" = 방학동 | "법정동" 데이터는 프론트로 부터 받지 않을 수도 있음.
input_district_name = "성북구"
input_legal_dong_name = None
input_building_use = "연립다세대"
input_amount = "20000" # 10,000 단위 처리 필요 (프론트에서)

# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
# print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

# 건물 가격 필터링
def filter_by_amount(df, building_amount):
    building_amount = int(building_amount)
    return df[df['amount'] < building_amount].copy()

# 건물 유형 필터링
def filter_by_building_type(df, building_type):
    return df[df['building_use'] == building_type].copy()


# 지역구 필터링
def filter_by_district(df, district_name):
    return df[df['district_name'] == district_name].copy()


# 전체 구 | 평단가 계산 | 건물 가격 단위(만원) | 건물 면적 단위 (m^2) | 모든 건물의 평균 단가 개발 계산 | price_per_sqm 컬럼 생성
def calc_price_per_sqm(df):
    df = df.copy()
    df['price_per_sqm'] = (
        df['amount'] * 10000
    ) / df['building_area']
    return df


# 법정동 연도별 평단가 평균 계산 | yearly_avg_price DataFrame 생성 | transaction_count, avg_price_per_sqm 컬럼 생성
def calc_yearly_avg_price(df):
    df = df.copy()
    return (
        df
        .groupby(['reception_year', 'district_name', 'legal_dong_name'])
        .agg(
            avg_price_per_sqm=('price_per_sqm', 'mean'),
            transaction_count=('price_per_sqm', 'count')
        )
        .reset_index()
        .sort_values(['legal_dong_name', 'district_name', 'reception_year'])
    )


# 포맷팅 round(0)으로 소수점 첫 째 자리에서 반올림 | price_per_sqm_format 컬럼 생성
def format_price_column(df):
    df = df.copy()
    df['price_per_sqm_format'] = (
        df['avg_price_per_sqm']
        .round(0)
        .map(lambda x: f"{int(x):,}")
    )
    return df

# 연도별 상승률 | yoy_change_rate 컬럼생성 | 앞 행(전년도)과의 평단가의 차이 %로 계산
def calc_yoy_change_rate(df):
    df = df.copy()
    df['yoy_change_rate'] = (
        df
        .groupby(['district_name', 'legal_dong_name'])['avg_price_per_sqm']
        .pct_change() * 100
    ).round(2)
    return df

# 전체 상승률 | 2025(데이터 마지막 년도)평단가 - 2022 년도(데이터 시작년도) 평단가의 차이 %로 계산
def calc_total_change_rate(df):
    df = df.copy()

    total_rate = (
        df
        .groupby(['district_name', 'legal_dong_name'])['avg_price_per_sqm']
        .agg(['first', 'last'])
        .reset_index()
    )

    total_rate['total_change_rate'] = (
        (total_rate['last'] - total_rate['first'])
        / total_rate['first'] * 100
    ).round(2)

    return df.merge(
        total_rate[['district_name', 'legal_dong_name', 'total_change_rate']],
        on=['district_name', 'legal_dong_name'],
        how='left'
    )

# 랭킹 계산
def apply_change_rank(df):
    df = df.copy()
    df['change_rank'] = (
        df
        .groupby('district_name')['total_change_rate']
        .rank(method='dense'     #1, 2, 3 (중복 순위 허용, 다음 순위 건너뛰지 않음)
              , ascending=False) # 상승률 높은 게 1위
        .astype(int)
    )
    return df

# 원본 데이터 할당
district_df = df_ret

# 입력받은 건물 가격이 있으면 원본 데이터를 건물 가격 언더로만 필터링 후 지역구로 필터링
# 건물 가격을 받지 않으면 원본 데이터를 지역구에서만 필터링
if input_amount is not None:
    district_df = filter_by_amount(district_df, input_amount)

# 건물 유형이 있으면 원본 데이터를 건물 유형에 맞게 필터링 후 지역구로 필터링
# 건물 유형이 없으면 원본 데이터를 지역구에서만 필터링
if input_building_use is not None:
    district_df = filter_by_building_type(district_df, input_building_use)

# 지역구 필터링
district_df = filter_by_district(district_df, input_district_name)

district_df = calc_price_per_sqm(district_df)

yearly_avg_price = calc_yearly_avg_price(district_df)
yearly_avg_price = format_price_column(yearly_avg_price)
yearly_avg_price = calc_yoy_change_rate(yearly_avg_price)
yearly_avg_price = calc_total_change_rate(yearly_avg_price)
yearly_avg_price = apply_change_rank(yearly_avg_price)



# 랭킹으로 정렬
yearly_avg_price = yearly_avg_price.sort_values(
    ['change_rank', 'reception_year']
    
)

# 법정동 유무에 따른 프론트 던지기?
if input_legal_dong_name is not None:
        yearly_avg_price = yearly_avg_price[
            yearly_avg_price['legal_dong_name'] == input_legal_dong_name
        ]

yearly_avg_price


,reception_year,district_name,legal_dong_name,avg_price_per_sqm,transaction_count,price_per_sqm_format,yoy_change_rate,total_change_rate,change_rank
50,2022,성북구,하월곡동,3.376609e+06,3,"3,376,609",NaN,187.20,1
51,2023,성북구,하월곡동,4.070803e+06,3,"4,070,803",20.56,187.20,1
52,2024,성북구,하월곡동,9.697768e+06,6,"9,697,768",138.23,187.20,1
33,2022,성북구,성북동,5.126613e+06,2,"5,126,613",NaN,79.47,2
34,2023,성북구,성북동,7.623988e+06,2,"7,623,988",48.71,79.47,2
35,2024,성북구,성북동,9.200690e+06,1,"9,200,690",20.68,79.47,2
36,2022,성북구,안암동2가,5.275974e+06,1,"5,275,974",NaN,76.06,3
37,2023,성북구,안암동2가,9.289041e+06,18,"9,289,041",76.06,76.06,3
25,2022,성북구,삼선동4가,5.080978e+06,2,"5,080,978",NaN,74.35,4
26,2025,성북구,삼선동4가,8.858859e+06,6,"8,858,859",74.35,74.35,4


## 2. 공영주차장 (Public Parkings)

In [4]:
# SQLAlchemy를 사용하여 데이터 가져오기
parkings = PublicParking.query.limit(5).all()
print(f"가져온 주차장 레코드 수: {len(parkings)}")

# DataFrame으로 변환
stmt = select(PublicParking)
df_pp = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_pp)}")
df_pp.head()

가져온 주차장 레코드 수: 5
총 레코드 수: 1875


,pp_id,parking_code,parking_name,address,parking_type,parking_type_name,operation_type,operation_type_name,phone_number,parking_status_available,...,bus_basic_time_min,bus_add_rate,bus_add_time_min,day_max_rate,lat,lng,share_parking_company_name,share_parking,share_parking_company_link,share_parking_etc
0,1,171721,세종로 공영주차장(시),종로구 세종로 80-1,NW,노외 주차장,1,시간제 주차장,02-2290-6566,1,...,0,0,0,30900,37.573403,126.975884,,N,,
1,2,171730,종묘주차장 공영주차장(시),종로구 훈정동 2-0,NW,노외 주차장,1,시간제 주차장,02-2290-6166,1,...,0,0,0,28800,37.571504,126.994969,,N,,
2,3,171900,훈련원공원 공영주차장(시),중구 을지로5가 40-3,NW,노외 주차장,1,시간제 주차장,02-3405-4597,1,...,0,0,0,39600,37.567400,127.003521,,N,,
3,4,172051,한강진역 공영주차장(시),용산구 한남동 728-27,NW,노외 주차장,1,시간제 주차장,02-795-6406,1,...,0,0,0,28800,37.539522,127.002580,,N,,
4,5,172065,용산주차빌딩 공영주차장(시),용산구 한강로2가 12-9,NW,노외 주차장,1,시간제 주차장,02-2290-6014,1,...,0,0,0,21600,37.534364,126.965418,,N,,
